# PANDAS: The functions apply and agg

The difference between the function apply, agg and tranform are quite obscure. In several cases we will get the same result, so we can use the indistinctively. 

## Initialise the environment

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [112]:
arr = np.array([1,3,4, 5,6,7])
s = pd.Series(arr)

## Get our data set


We will analyse data of videogame **sales** during the last decades

In [12]:
df = pd.read_csv("vgsales.csv")
df.head()

,Unnamed: 0,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46
1,1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77
2,2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31
3,3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96
4,4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00


In [87]:
df.shape

(16598, 12)

Let us observe the attributes

In [13]:
df.dtypes

Unnamed: 0       int64
Rank             int64
Name            object
Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales       float64
EU_Sales       float64
JP_Sales       float64
Other_Sales    float64
dtype: object

## Tha apply function vs the agg function

Let us answer to the following questions:

#### 1. What is the total number of sales (for all regions) per game

Let us use the normal panda operations to compute this value

In [85]:
t = time.process_time()
data1 = df.copy()
data1["Global_Sales"] = data1["NA_Sales"] + data1["EU_Sales"] + data1["JP_Sales"] + data1["Other_Sales"]
print(time.process_time() - t)

0.01090199999998731


Let us use the apply function

In [86]:
t = time.process_time()
def total(line):
    return line.NA_Sales + line.EU_Sales + line.JP_Sales + line.Other_Sales

data2 = df.copy()
data2.loc[:, "Global_Sales"] = data1.apply(total, axis=1)
print(time.process_time() - t)

1.7178399999999954


Another way:

In [90]:
t = time.process_time()
def total(line):
    return line.sum()

data3 = df.copy()
data3.loc[:, "Global_Sales"] = data1[["NA_Sales","EU_Sales","JP_Sales","Other_Sales"]].apply(total, axis=1)
print(time.process_time() - t)

3.114028000000019


let us use the agg function

In [93]:
t = time.process_time()
def total(line):
    return line.NA_Sales + line.EU_Sales + line.JP_Sales + line.Other_Sales

data4 = df.copy()
data4.loc[:, "Global_Sales"] = data1.agg(total, axis=1)
print(time.process_time() - t)

3.686616000000015


In [35]:
df = data4.copy()

#### 2. What is the percentage of sales per region per game

In [94]:
t = time.process_time()
data1 = df.copy()
data1.loc[:, "Percentage_JP"] = (data1["JP_Sales"]/data1["Global_Sales"]).round(2)*100
data1.loc[:, "Percentage_EU"] = (data1["EU_Sales"]/data1["Global_Sales"]).round(2)*100
data1.loc[:, "Percentage_NA"] = (data1["NA_Sales"]/data1["Global_Sales"]).round(2)*100
print(time.process_time() - t)

0.03122600000000375


In [99]:
t = time.process_time()
def percentage_jp(line):
    if line.Global_Sales > 0:
        return line.JP_Sales/line.Global_Sales

def percentage_na(line):
    if line.Global_Sales > 0:
        return line.NA_Sales/line.Global_Sales

data2 = df.copy()
data2.loc[:, "Percentage_JP"] = data2.apply(percentage_jp, axis=1)
data2.loc[:, "Percentage_NA"] = data2.apply(percentage_na, axis=1)
print(time.process_time() - t)

2.802677999999986


In [102]:
t = time.process_time()
def percentage(line, region):
    if line.Global_Sales > 0:
        return line[region]/line.Global_Sales

data2 = df.copy()
data2.loc[:, "Percentage_JP"] = data2.apply(percentage, axis=1, region="JP_Sales")
data2.loc[:, "Percentage_NA"] = data2.apply(percentage, axis=1, region="NA_Sales")
print(time.process_time() - t)

2.647564000000017


In [104]:
t = time.process_time()
def percentage(line):
    if line.Global_Sales > 0:
        return {"Percentage_JP": line.JP_Sales/line.Global_Sales, "Percentage_NA": line.NA_Sales/line.Global_Sales, "Percentage_EU":line.EU_Sales/line.Global_Sales}

data2 = df.copy()
result= data2.apply(percentage, axis=1, result_type="expand")
data2 = data2.join(result)
print(time.process_time() - t)
data2

7.559401999999977


,Unnamed: 0,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Percentage_JP,Percentage_NA,Percentage_EU
0,0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,0.045564,0.501450,0.350737
1,1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,0.169235,0.722664,0.088966
2,2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.83,0.105777,0.442367,0.359475
3,3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,0.099394,0.477273,0.333636
4,4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.38,0.325685,0.359146,0.283301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16593,16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01,0.000000,1.000000,0.000000
16594,16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01,0.000000,1.000000,0.000000
16595,16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
16596,16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01,0.000000,0.000000,1.000000


#### 3. What are the mean sales value per games producer company

In [105]:
## For nintendo for instance
data1 = df.copy()
nintendo = data1[data1.Publisher == "Nintendo"]

japan = nintendo.JP_Sales.mean()
print(japan)
americ = nintendo.NA_Sales.mean()
print(americ)
europe = nintendo.EU_Sales.mean()
print(europe)

0.6478236130867703
1.161977240398293
0.5956472261735423


Apply

In [106]:
t = time.process_time()
def my_mean(data):
    return data.mean()

mean_vals = nintendo[["JP_Sales", "NA_Sales", "EU_Sales"]].apply(my_mean)
print(time.process_time() - t)
print(mean_vals)

0.005186000000009017
JP_Sales    0.647824
NA_Sales    1.161977
EU_Sales    0.595647
dtype: float64


Agg

In [109]:
t = time.process_time()
def value(column):
    return column[0]
    
nintendo.agg({"NA_Sales": "mean", "JP_Sales": ["mean", "min", "max", "sum", "count", value], "EU_Sales": ["mean"]})


,NA_Sales,JP_Sales,EU_Sales
count,NaN,703.000000,NaN
max,NaN,10.220000,NaN
mean,1.161977,0.647824,0.595647
min,NaN,0.000000,NaN
sum,NaN,455.420000,NaN
value,NaN,3.770000,NaN


TODO: Use groupby to calculate the mean value per Publisher